This code references: https://github.com/davidyslu/USTC-TK2016/blob/master/3_Session2Png.py

Importing Libraries

In [3]:
import os
from array import *
from scapy.all import rdpcap
import json
import numpy
from PIL import Image
import binascii

Matrix from Pcap Function

In [ ]:
def get_matrix_from_pcap_packets(filename, width, ethernet_version):
    packets = rdpcap(filename)
    
    hexst = b''
    
    if ethernet_version[1]:
        zeros = b'0'*28

    meta_data = []
    
    for i, packet in enumerate(packets):
        hex_content = binascii.hexlify(bytes(packet))
        
        '''No Empty Packets'''
        if len(hex_content) > len(b'ffffffffffff70cf49a5c0219000'):
            
            if ethernet_version[0]:
                meta_data.append(len(hexst) // 2)
                hexst = hexst + hex_content
                
            elif ethernet_version[1]:
                hex_content = hex_content[28:]
                hex_content = zeros + hex_content
                meta_data.append(len(hexst) // 2)
                hexst = hexst + hex_content
            
            elif ethernet_version[2]:
                hex_content = hex_content[28:]
                meta_data.append(len(hexst) // 2)
                hexst = hexst + hex_content
            
    fh = numpy.array([int(hexst[i:i+2], 16) for i in range(0, len(hexst), 2)])
    max_dim = (width, width)
    padding = (max_dim * max_dim) - len(fh)
    
    if padding > 0:
        fh = numpy.append(fh, numpy.zeros(padding, dtype=numpy.uint8))
        
    fh = numpy.reshape(fh, (width, width))
    fh = numpy.reshape(fh, (-1, width))
    fh = numpy.uint8(fh)
    
    return fh, meta_data

Converting Pcap Sessions to Images

In [ ]:
pcaps_path = "path/to/pcapfiles/directory/"
metadata_path = "path/to/store/generated/metadata/"
images_path = "path/to/store/generated/images/"

pcaps_names = os.listdir(pcaps_path)

count = 0

image_size = 32

'''     [ keep_ethernet, replace_ethernet_with_0s, remove_ethernet]      '''
ethernet_version = [True, False, False]

for file in pcaps_names:
    count += 1
    print(str(count))
    
    file_name = file.replace(".pcap", "")
    
    fh, metadata = get_matrix_from_pcap_packets(pcaps_path+file, image_size, ethernet_version)
    
    image = Image.fromarray(fh)
    print(f"{file_name}.png")
    
    with open(metadata_path+file_name+".json", "w") as md_file:
        json.dump(metadata, md_file)
    
    image.save(f"{images_path}{file_name}.png")
    